## imports

In [ ]:
import json
import warnings
from datetime import datetime, timedelta
from io import BytesIO, StringIO

import git
import pandas as pd
import PIL

warnings.filterwarnings("ignore")

---
## fetch commits

In [ ]:
YEAR = 2024
CSV_PATH = f"CTFTIME_{YEAR}.csv"

repo = git.Repo('./data')

top_teams = []
for commit in repo.iter_commits(paths=CSV_PATH):
    # Read file contents from each commit
    file_contents = (commit.tree / CSV_PATH).data_stream.read()
    df = pd.read_csv(StringIO(file_contents.decode()))

    # Add a 'weeks' column indicating the week number of the commit
    #week_number = commit.committed_datetime.isocalendar()[1]
    week_number = (commit.committed_datetime - timedelta(days=14)).isocalendar()[1]
    df.loc[:, 'weeks'] = f'week {week_number:02}'

    top_teams.append(df)

# Concatenate all weekly DataFrames in chronological order
df = pd.concat(top_teams[::-1], axis=0, ignore_index=True)

---
## Clean the data and merge latest country information
Remove duplicates based on 'rank' and 'weeks', and ensure each team has the latest 'country' data by week. 

In [ ]:
df = df.drop_duplicates(subset=['rank', 'weeks'], keep='last')

# Get the latest country information for each team by name
latest_countries = df.sort_values('weeks').groupby('Name').last()['country']

# Merge the latest country data back into the main DataFrame
df = df.drop(columns='country').merge(latest_countries, on='Name', how='left')

# some rename
df.loc[:,'Name'] = df['Name'].replace('💦\u200b', 'BlueWater')

---
## Filter to include only teams that were in the top N at least once

In [ ]:
top_n=10
topTeams = df[df['rank'] <= top_n]['Name'].unique()
selected_df = df[df['Name'].isin(topTeams)]

---
## Pivot data for Flourish Bar chart race
- https://app.flourish.studio/@flourish/bar-chart-race

In [ ]:
# Pivot the data by 'weeks' to have scores for each week as separate columns
df_pivot = selected_df.pivot_table(
    index=['Name', 'country'], 
    columns='weeks', 
    values='score', 
    fill_value=0  # Set missing scores to 0
).reset_index()
df_pivot.columns.name = None  # Remove the multi-index column grouping name

# Save the pivoted DataFrame to CSV
df_pivot.to_csv('df_pivot.csv', encoding='utf-8', index=False, header=True)

---
# with [Bar Chart Race](https://github.com/dexplo/bar_chart_race)

In [ ]:
from itertools import cycle

teamName = selected_df.drop_duplicates(subset='Name', keep='first')['Name'].tolist()
colors = [
    'red', 'blue', 'green', 'yellow', 'cyan', 'magenta', 'black', 'white', 
    'gray', 'orange', 'purple', 'pink', 'brown', 'lime', 'olive', 'gold', 
    'teal', 'navy', 'coral', 'salmon', 'indigo', 'violet']

[*zip(teamName, itertools.cycle(colors))]

In [ ]:
# pip install bar_chart_race
import bar_chart_race as bcr

df_pivot = selected_df.pivot(index='weeks', columns='Name', values='score').fillna(0)

bcr.bar_chart_race(
    df=df_pivot,
    title='Top 10 Teams Race in 2024',
    n_bars=10,  # Display top 10 individuals
#     shared_fontdict={
#         'family': 'FreeMono' # FreeMono, FreeSerif
#     },
    filename=f'CTF Ranking Race - {YEAR} Insights.mp4',
    orientation='h',  # Horizontal bars
    sort='desc',  # Sort in descending order
    bar_size=0.9,
    cmap='dark24', # tab20, Light24, dark24
    period_length=500  # Speed of animation (lower is faster)
)

---
## with matplotlib

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

top_teams = [team[team['rank']<=10] for team in top_teams[::-1]]

def update(frame):
    plt.cla()  # Clear the current axes
    df = top_teams[frame]  # Get the DataFrame for the current year
    teams = df['Name']
    performance = df['score']
    plt.barh(teams, performance, color='skyblue')
    plt.xlabel('score')
    plt.ylabel('Name')
    for i, value in enumerate(df['score']):
        plt.text(value, i, df.loc[i, 'Name'], ha='center', va='center', fontsize=10)
        plt.text(value/2, i, f'{value}', ha='center', va='center', fontsize=10, color='red')
    plt.title(f'Top 10 Teams for week {frame+1}')
    plt.gca().invert_yaxis()  # Invert y-axis to have the top team at the top


plt.rcParams["font.size"] = 10
plt.rcParams["font.family"] = "FreeSerif"

fig = plt.figure(figsize=(16, 9), dpi=1920/16)
ani = FuncAnimation(fig, update, frames=len(top_teams), interval=1000)  # interval is in milliseconds

ani.save('competition_animation.mp4', writer='ffmpeg')

plt.show()

---
## with [raceplotly](https://github.com/lucharo/raceplotly)

In [ ]:
# pip install raceplotly
from raceplotly.plots import barplot

my_raceplot = barplot(selected_df,
                      item_column='Name',
                      value_column='score',
                      time_column='weeks',
                      top_entries=10)

my_raceplot.plot(title = f'Top 10 Teams in {YEAR}',
                 item_label = 'Top 10 Teams',
                 value_label = 'Score',
                 frame_duration = 800)

In [ ]:
# pip install kaleido
fig = my_raceplot.fig

frames = []
for s, fr in enumerate(fig.frames):
    # set main traces to appropriate traces within plotly frame
    fig.update(data=fr.data)
    # move slider to correct place
    fig.layout.sliders[0].update(active=s)
    # generate image of current state
    frames.append(PIL.Image.open(BytesIO(fig.to_image(format="png"))))

for i in range(2):
    frames.append(frames[-1])

# create animated GIF
frames[0].save(
        "raceplotly.gif",
        save_all=True,
        append_images=frames[1:],
        optimize=False,
        duration=1000,
        loop=0,
    )

---
# Font issue

Installing Font
```bash
sudo cp /mnt/c/path/to/VictorMono.ttf /usr/local/share/fonts/
sudo fc-cache -fv # reload system font cache
fc-list | grep "Victor Mono"
```

delete matplotlib font cache
```bash
rm -fv $HOME/.cache/matplotlib/fontlist*.json
```

In [ ]:
import shutil, matplotlib
cachedir = matplotlib.get_cachedir()
shutil.rmtree(cachedir)
print(f'delete {cachedir=}')

In [ ]:
# List of available font names
from matplotlib import font_manager

# print(font_manager.findSystemFonts(fontpaths=None, fontext='ttf'))
print(font_manager.get_font_names())

In [ ]:
font_manager.findfont('FreeSerif', rebuild_if_missing=True)

In [ ]:
# method 1
plt.rcParams['font.family'] = 'FreeSerif'

# method 2
font_path = '/path/to/your/emoji-font.ttf'
font_manager.fontManager.addfont(font_path)
plt.rcParams['font.family'] = font_manager.FontProperties(fname=font_path).get_name()

### test emoji support

In [ ]:
import matplotlib.pyplot as plt

for selected_font in font_manager.get_font_names():
    plt.rcParams['font.family'] = selected_font
    plt.figure(figsize=(6, 4))
    plt.plot([1, 2, 3], [3, 2, 5], label="Growth 📈")
    plt.xlabel("Time 🕒")
    plt.ylabel("Value 💹")
    plt.title(f"Trend Over Time 🔥 with {selected_font}")
    plt.legend()
    output_filename = f"img/sample_plot_with_emoji_{selected_font.replace(' ', '_')}.png"
    plt.savefig(output_filename)
    #plt.show()
    print(f"Plot saved as '{output_filename}' with font '{selected_font}'")